# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from ate_models.ATE_BERT_Dropout_BiLSTM_Linear import ATE_BERT_Dropout_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_MAMS_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
1,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
2,The decor is not special at all but their food...,"[The, decor, is, not, special, at, all, but, t...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."
3,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"when tables opened up, the manager sat another...","[when, tables, opened, up, ,, the, manager, sa...","[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
BERT_FINE_TUNED_PATH = '../../../results/ATE/MAMS/models/bert_fine_tuned.pth'

In [11]:
MODEL_OUTPUT = '../../../results/ATE/MAMS/models/bert_fine_tuned_dropout_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/MAMS/stats/bert_fine_tuned_dropout_bilstm_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ATE_BERT_Dropout_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), dropout=0.3, bilstm_in_features=256, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/2237, Loss: 1.1951433420181274
Epoch: 0/2, Batch: 223/2237, Loss: 0.07952836900949478
Epoch: 0/2, Batch: 446/2237, Loss: 0.10330259799957275
Epoch: 0/2, Batch: 669/2237, Loss: 0.07506105303764343
Epoch: 0/2, Batch: 892/2237, Loss: 0.06397013366222382
Epoch: 0/2, Batch: 1115/2237, Loss: 0.054204873740673065
Epoch: 0/2, Batch: 1338/2237, Loss: 0.0828690305352211
Epoch: 0/2, Batch: 1561/2237, Loss: 0.045660171657800674
Epoch: 0/2, Batch: 1784/2237, Loss: 0.037886153906583786
Epoch: 0/2, Batch: 2007/2237, Loss: 0.01620856113731861
Epoch: 0/2, Batch: 2230/2237, Loss: 0.04230530932545662
Epoch: 1/2, Batch: 0/2237, Loss: 0.024709636345505714
Epoch: 1/2, Batch: 223/2237, Loss: 0.0064264643006026745
Epoch: 1/2, Batch: 446/2237, Loss: 0.041651055216789246
Epoch: 1/2, Batch: 669/2237, Loss: 0.041600558906793594
Epoch: 1/2, Batch: 892/2237, Loss: 0.052952591329813004
Epoch: 1/2, Batch: 1115/2237, Loss: 0.009115450084209442
Epoch: 1/2, Batch: 1338/2237, Loss: 0.0300727

In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.993164,0.993164,0.967493,0.993164,0.989120,0.993164,0.978096,453.174581
1,0.994098,0.994098,0.972322,0.994098,0.990406,0.994098,0.981217,437.798032
2,0.992646,0.992646,0.964468,0.992646,0.989560,0.992646,0.976722,437.227031
3,0.992693,0.992693,0.966554,0.992693,0.987819,0.992693,0.976976,438.053217
4,0.992962,0.992962,0.968637,0.992962,0.984984,0.992962,0.976671,436.140970
5,0.992916,0.992916,0.970344,0.992916,0.984887,0.992916,0.977475,435.893028
6,0.993717,0.993717,0.976571,0.993717,0.984014,0.993717,0.980268,437.232998
7,0.992397,0.992397,0.964090,0.992397,0.990211,0.992397,0.976828,436.384657
8,0.993425,0.993425,0.972847,0.993425,0.984210,0.993425,0.978471,435.891000
9,0.992985,0.992985,0.967598,0.992985,0.987767,0.992985,0.977500,437.152997


In [20]:
results.to_csv(STATS_OUTPUT)